# NetGAN

Introduced by A. Bojchevski et al. in 2018, NetGAN is a graph generative model able to mimic real-world networks.

In NetGAN the problem of learning the graph topology is formulated as learning the distribution of biased random walks over the graph: a **generator** $G$ (here a *stochastic* NN with discrete output samples) learns to generate random walks that are plausible in the real graph, while the **discriminator** $D$ then has to distinguish them from the true ones that are sampled from the original graph.

The advantages of such an approach are multiple: NetGAN is able of exploiting the sparsity of real world graphs by operating on RWs and considering only the non-zero entries of the adj matrix, RWs are invariant under node reordering and this property overcomes a problem that has limited graph generation scalability with other approaches, ...

## The model

Once again, the core idea is capturing the topology of a graph through learning a distribution over the random walks.

Given an input binary adj matrix $A = \{0,1\}^{N\times N}$, we first sample a set of RWs of length $T$ from $A$. This set serves as a **training set** for our model and the RWs are sampled through the *biased second-order RW sampling strategy* (Grover & Leskovec, 2016) as it better captures both global and local structures of the graph.

The goal of the generator is to generate synthetic RWs that are plausible in the input graph while simultaneously the discriminator learns to distinguish the synthetic RWs from the real ones that come from the training set. At any point of the training it is possible to use $G$ to generate a set of RWs which can then be used to produce an adj matrix of a new generated graph.

### Sampling RWs

## Architecture
### Generator

$G$ defines an implicit probabilistic model for generating RWs $(v_1, v_2, ..., v_T)\sim G$, and we model it as a *sequential process* based on a neural network $f_\theta$ parametrized by $\theta$. At each step $t\in\{0,...,T\}$, $f_\theta$ produces two values: the probability distribution over the next node to be sampled, parametrized by logits $p_t$, and the current memory state of the model $m_t$. The next node $v_t$ represented as a one-hot encoded vector of $dim(v_t)=N$, is sampled from a categorical distribution $v_t\sim Cat(\sigma(p_t))$ where $\sigma(\cdot)$ here represents the **softmax** function:

$$\sigma(p_t^i)= \frac{e^{p_t^i}}{\sum_{i=1}^{N}{e^{p_t^i}}}$$

$v_t$ together with $m_t$ is then passed into $f_\theta$ at the step $t+1$. A latent encoded variable $z$ is drawn from a multivariate standard normal distribution and passed through a parametric function $g_{\theta'}$ to initialize $m_0$.
The entire generative process of $G$ is therefore summarized as follows:


Initialization:

- Step 1: $\hspace{3.7cm} z\sim\mathcal{N(0,I)}$
- First state:                      $\hspace{3cm} m_0 = g_{\theta'}(z)$
  
Repeat $T$ times:  

- NN sampling:                      $\hspace{3cm} (p_1, m_1) = f_{\theta}(m_0, 0)$
- Node sampling:                    $\hspace{3cm} v_1\sim Cat(\sigma(p_1))$
- NN sampling:                      $\hspace{3cm} (p_2, m_2) = f_{\theta}(m_1, v_1)$
- Node sampling:                    $\hspace{3cm} v_2\sim Cat(\sigma(p_2))$
- ...
- NN sampling:                      $\hspace{3cm} (p_T, m_T) = f_{\theta}(m_{T-1}, v_{T-1})$
- Node sampling:                    $\hspace{3cm} v_T\sim Cat(\sigma(p_T))$

Intuitively, $f_{\theta}$ is a **recurrent NN** as it takes as input the previous state of the network. In the original NetGAN work, $f_\theta$ was modelled as an LSTM.

For an LSTM, the *memory state* is represented by a **cell state** $C_t$ and by a **hidden state** $h_t$. The latent variable $z$ goes then through two separate layers with a `tanh` activation which are then used to sample $(C_0, h_0)$.

Of course we should ask ourselves why do we use a model with a memory while RWs are markovian processes (2nd order markovian processes for biased RWs). In short, having longer walks with memory helps the model to learn topology and general patterns in the data while preserving computational feasability.

After each time step, to generate the next node in the RW, the NN $f_\theta$ should output $p_t$ of length $N$. However, when dealing with big graphs this lead to unnecessary computational overhead. To tackle this issue, the LSTM outputs $o_t \in \mathbb{R}^H$, with $H<<N$ which is then up-projected to $\mathbb{R}^N$ using the matrix $W_{up}\in\mathbb{R}^{H\times N}$. This is pivotal for handling large-scale graphs. There are other caveats for sampling $v_t$ from a categorical distribution as it is a non-differentiable operation but those will not be discussed here. 
Of course, once the node is sampled, likewise the previous operation for $p_t$, it needs to be projected back to a lower-dimensional representation before feeding it to the LSTM. This is achieved through the down-projection matrix $W_{down}\in\mathbb{R}^{N\times H}$ 

### Discriminator

The discriminator D is based on the standard LSTM architecture: at every time step $t$, a one-hot encoded vector $v_t$, denoting the node at the current position, is fed as an input.
After processing the entire sequence of $T$ nodes, the discriminator outputs a single score which represents the probability of the random walk being real.

## Training
### Wasserstein GAN

We train our model based on the Wasserstein GAN (WGAN) framework, the generator's parameters $\{\theta,\theta'\}$ are trained using SGD with Adam, weights are regularized with $L^2$ penalty.

### Early stopping

We are interested in generalizing the input graph, this means checking how closely the generated graphs resemble the original one.
Early stopping could be based on what were called in the original article `VAL-CRITERION` or `EO-CRITERION`.

- `VAL-CRITERION`: The idea is that during training we keep a sliding window of the RWs generated in the last 1000 iterations and used them to construct a *matrix of transition counts* which is then used to evaluate **link prediction performance** (AP and ROC scores) on a validation set. This early stoppig strategy stops the training whenever the validation performance stops improving.

- `EO-CRITERION`: this strategy makes NetGAN very flexible and allows the user control over the graph generation. We stop training when we achieve a user specified overlap between the generated graphs. The idea is that based on the user's task, he can choose to generate graphs with either small or large overlap with the original while mantaining structural similarity which will lead to graphs that either generalize better or are close replicas, yet still capture the properties of the original.

## Assembling the adj matrix

After the training, we use $G$ to construct a score matrix $S$ of the **transition counts**, i.e. we count how often a certain node occures in the set of generated RWs typically by using a much larger number of RWs than for early stopping, e.g. $\sim 500K$.

In order to convert the count matrix $S$ to the proposed binary adj matrix $\hat{A}$, we need to perform a few steps. First, we **symmetrize** $S$ by setting $s_{ij}=s_{ji}=max{(s_{ij},s_{ji})}$ as we are generating unweighted graphs. 

In addition, as we do not explicitely control the starting node of the RWs generated by $G$, some high degree-nodes will likely be overrepresented. Therefore, a simple binarization strategy (thresholding or choosing top-k entries) might lead to leaving out low-degree nodes and producing singletons.
To address this we choose the following strategy:
1. We ensure that every node $i$ has at least one edge by sampling a neighbour $j$ with probability $p_{ij}=\frac{s_{ij}}{\sum_{v=1}^{N}{s_{iv}}}$. If an edge was already sampled before, we repeat the procedure
2. We continue sampling edges without replacement using for each edge $(i,j)$ the probability $p_{ij}=\frac{s_ij}{\sum_{u,v}{s_{uv}}}$ until we reached the same amount of edges of the original graph.